# Food Vision

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import datetime
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
!nvidia-smi -L
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/refs/heads/main/extras/helper_functions.py

## Load the Dataset from Tensorflow Datasets

In [ ]:
(train_data, test_data), ds_info = tfds.load(
    name="food101",
    split=["train", "validation"],
    shuffle_files=True,
    as_supervised=True,
    with_info=True
)

In [ ]:
len(train_data), len(test_data)

In [ ]:
print(f"Dataset info: {ds_info}")

In [ ]:
class_names = ds_info.features['label'].names

In [ ]:
sample = train_data.take(1)

In [ ]:
for a,b in sample:
  print(f'Data Type of tensor {a.dtype}')
  print(f'Data Shape of tensor {a.shape}')
  print(f'Label {b} which means {class_names[b]}')
  plt.figure(figsize=(6,6))
  plt.title(f'{class_names[b]}')
  plt.imshow(a)


## Pre-Process the Data

Currently we have tensors of **uint8** datatype. I plan on running mixed precision training, therefore we'd need **float16/32** bit tensors. They're also of various shapes, and non - normalised, which models tend to prefer. Let's fix this, while also implementing batching.

In [ ]:
def pre_process_image(image,label, new_shape=[224,224]):
  image = tf.image.resize(image, new_shape)
  new_image = tf.cast(image, tf.float32)
  new_image = new_image/255.0
  return new_image, label
a,b = pre_process_image(a,b)
print(a.dtype)
print(a.shape)
plt.imshow(a)

In [ ]:
# Batching the data, and then running shuffle again
train_data = train_data.map(map_func=pre_process_image, num_parallel_calls=tf.data.AUTOTUNE)
# shuffle and prefetch, prefetching is getting the next segment of data to shuffle while you shuffle the current one
# the prefetching is on CPU, while map is on the GPU. the AUTOTUNE parrallel call enables parallelization
train_data = train_data.shuffle(buffer_size=1000).batch(batch_size=32, num_parallel_calls=tf.data.AUTOTUNE).prefetch(buffer_size=tf.data.AUTOTUNE)

test_data = test_data.map(map_func=pre_process_image, num_parallel_calls=tf.data.AUTOTUNE)
test_data = test_data.shuffle(buffer_size=1000).batch(batch_size=32, num_parallel_calls=tf.data.AUTOTUNE).prefetch(buffer_size=tf.data.AUTOTUNE)

## Creating some callbacks, in case we need to investigate further down the line

In [ ]:
# Creating a Tensorboard Callback, just to log metrics during training such that we can look later on Tensorboard at them

def create_tensorboard_callback(dir_name, experiment_name):
  log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  tensorboard_callback = tf.keras.callbacks.TensorBoard(
      log_dir=log_dir
  )
  print(f"Saving TensorBoard log files to: {log_dir}")
  return tensorboard_callback


In [ ]:
# Model Checkpoint to save weights during training
checkpoint_path = 'checkpoint_path/cp.weights.h5'
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,monitor='val_acc',
                                                      verbose=0,save_best_only=True,save_weights_only=True)

## Using Mixed Precision Training

In [ ]:
# Using mixed precision training speeds up model training massively. By default, most tensor parameters are stored
# as 32 bit. By setting some such as activations, and CNN window weights as 16bit we can massively improve
# efficiency (hlaving our space). We still keep weights and biases as 32bit though
!nvidia-smi -L
# We're using Tesla T4 GPU. This has a compute capability of 7.5, we need 7 so we're all good to go

In [ ]:
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16') # making sure we've mixed 16 and 32 bit

## Build the Feature Extraction Model

In [ ]:
#data augmentation - the benefit of incorporating it into the model is that it'll run on the gpu which is quicker

from tensorflow.keras import layers

data_aug = tf.keras.Sequential([layers.RandomFlip('vertical'),
                                layers.RandomHeight(0.2),
                                layers.RandomWidth(0.2),
                                layers.RandomRotation(0.2),
                                layers.RandomZoom(0.2)])

In [ ]:
# We're gonna use EficcientNet B0 for this one
from tensorflow.keras import layers
from tensorflow.keras import preprocessing

# Define input shape
input = layers.Input(shape=(224,224,3), name='input_layer')
input = data_aug(input)

#Instntiate Base Model
base = tf.keras.applications.EfficientNetB0(include_top=False)(input)
base.trainable= False

# Global Pooling 2D for a Feature Vector
pooling = layers.GlobalAveragePooling2D(name='pooling_layer')(base)

# Output
output = layers.Dense(len(class_names), activation='softmax', dtype = 'float32', name = 'output')(pooling)

# create the model and compile
model = tf.keras.Model(inputs=input, outputs=output)
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss = tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics = ['accuracy'])


In [ ]:
for layer in model.layers: # just double checking
  print(f'Layer Name {layer.name}', end = ', ')
  print(f'Layer Type {layer.dtype}', end = ', ')
  print(f'Layer Policy {layer.dtype_policy}')
  print('---')
for layer in model.layers[1].layers:
  print(f'Layer Name {layer.name}', end = ', ')
  print(f'Layer Type {layer.dtype}', end = ', ')
  print(f'Layer Policy {layer.dtype_policy}')
  print('---')

In [ ]:
food_vision_history = model.fit(train_data, batch_size=32, epochs=5,
          callbacks=[model_checkpoint, create_tensorboard_callback('Food_Vision', 'Iteration_1')],
          steps_per_epoch=len(train_data), validation_data=test_data, validation_steps=int(len(test_data)))


## Evaluate Model on all Test Data

In [ ]:
model.evaluate(test_data) # Let's see how the model does on the entireity of the test data
# Also, lets save it before we fine tune it
model.save('Food_Vision_model_non_fine_tuned.keras')

## Fine Tuning by unfreezing EfficientNet Layers

In [ ]:
for layer in model.layers[1].layers:
  layer.trainable = True


In [ ]:
for layer in model.layers:
  print(f'layer trainabale  = {layer.trainable}')

In [ ]:
fine_tuned_food_vision_history = model.fit(train_data, batch_size=32, epochs=10, initial_epoch=5,
          callbacks=[model_checkpoint, create_tensorboard_callback('Food_Vision', 'Iteration_2')],
          steps_per_epoch=len(train_data), validation_data=test_data, validation_steps=int(0.15*len(test_data)))


## Evaluate fine tuned model

In [ ]:
model.evaluate(test_data)
model.save('fine_tuned_model')

## Plot Model Histories

In [ ]:

# Combining training histories and visualize 
import pandas as pd
import matplotlib.pyplot as plt


df_1 = pd.DataFrame(food_vision_history.history)
df_2 = pd.DataFrame(fine_tuned_food_vision_history.history)
total_history = pd.concat([df_1, df_2], axis=0).reset_index(drop=True)

# Plot accuracy and loss curves
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.plot(total_history['accuracy'], label='Train Accuracy')
plt.plot(total_history['val_accuracy'], label='Val Accuracy')
plt.title('Model Accuracy')
plt.legend()

plt.subplot(1,2,2)
plt.plot(total_history['loss'], label='Train Loss')
plt.plot(total_history['val_loss'], label='Val Loss')
plt.title('Model Loss')
plt.legend()
plt.show()


In [ ]:

# Evaluate the model on the test dataset 
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import seaborn as sns

# Get test data predictions
y_true = np.concatenate([y for x, y in test_data], axis=0)
y_pred_probs = model.predict(test_data, verbose=1)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true_labels = np.argmax(y_true, axis=1)

# Classification report
print("Classification Report:\n")
print(classification_report(y_true_labels, y_pred))

# Confusion matrix
cm = confusion_matrix(y_true_labels, y_pred)
plt.figure(figsize=(10,8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()


In [ ]:

# Visualize sample predictions
import random
class_names = list(test_data.class_indices.keys())

plt.figure(figsize=(12, 12))
for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    img_batch, label_batch = test_data.next()
    img = img_batch[0]
    true_label = np.argmax(label_batch[0])
    pred_prob = model.predict(img_batch)[0]
    pred_label = np.argmax(pred_prob)

    plt.imshow(img)
    color = "green" if pred_label == true_label else "red"
    plt.title(f"T: {class_names[true_label]}\nP: {class_names[pred_label]}", color=color)
    plt.axis("off")
plt.tight_layout()
plt.show()
